## Importando as bibliotecas

- TensorFlow: biblioteca para processamento de dados e contrução de CNN
- JSON: biblioteca para codificação e decodificação de no formatpo JavaScript Object Notation

In [33]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import json
from os import path, getcwd
import sys

## Definindo a localização dos dados

Verifica se o código está sendo rodado no Google Colab e, se sim, monta o drive no ambiente de execução e aponta a pasta raiz do projeto

In [34]:

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive")
    BASE_PATH = "/content/drive/MyDrive/classification-of-medical-images-using-cnn/"
else:
    BASE_PATH = path.abspath(path.join(getcwd(), ".."))

Armazena o local onde os dados de treino e validação estão presentes e onde o modelo deve salvar o modelo já treinado, os pesos usados por ele e o histórico de treinamento

In [35]:
TRAIN_DIR = path.join(BASE_PATH, "data", "train")
VAL_DIR = path.join(BASE_PATH, "data", "val")
MODELS_PATH = path.join(BASE_PATH, "models", "xray_images.keras")
MODEL_WEIGHTS_PATH = path.join(BASE_PATH, "models", "xray_images.weights.h5")
RESULT_PATH = path.join(BASE_PATH, "results", "xray_images.json")

## Definição de parâmetros gerais

- `IMAGE_SIZE`: tamanho para o qual as imagens serão resimensionadas para que o modelo possa analisar
- `BATCH_SIZE`: tamanho do lote de imagens que serão analisadas pelo modelo a cada interação
- `EPOCHS`: número de vezes que o modelo analisará todas as imagens do conjunto de treinamento

In [36]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

## Carregamento do dataset

- `image_size`: resimensiona as imagens
- `batch_size`: define o número de imagens por lote
- `label_mode`: separa todas as imagens em duas classes

In [37]:
train_data = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, label_mode="binary"
)

val_data = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, label_mode="binary"
)

Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


##  Pré processamento das imagens

#### Noramalização

- `normalization_layer`: deffine para que os pixels da imagem sejam normalizados para valores entre 0 e 1

- Realiza a normalização nos conjuntos de treino e de validação

In [38]:
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))

#### Denfinindo e aplicando padrões de aumentação

- `RandomFlip`: inverte a imagem na horizontal aleatoriamente
- `RandomRotation`: gira a imagem aleatoriamente (0.05 equivale a até 5% da circunferência aprox. 18 graus)
- `RandomZoom`: amplia ou reduz na imagem aleatoriamente (0.1 equivale a até 10% do tamanho da imagem)

In [39]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.05),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

### Data augmentation

Realiza o data augmentation apenas no conjunto de teste

In [40]:
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

## Otimização do desempenho

O AUTOTUNE permite que a biblioteca do TensorFlow decida sozinho quantos dados carregar com antecedência, quando carregar e quanto de paralelismo usar, para se ajustar automaticamente para o melhor desempenho no hardware disponível

In [41]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

## Carregamento do Modelo Pré-treinado (transfer learning)

- Carrega o modelo base ResNet50 pré-treinada no Imagenet

- Congela as camadas da rede base

In [42]:
base_model = tf.keras.applications.ResNet50(
    weights="imagenet", include_top=False, input_shape=(224, 224, 3)
)

base_model.trainable = False

## Construção do modelo final

Diz ao modelo que ele receberá imagens de 224X224 pixels com 3 canais de cor

In [43]:
inputs = tf.keras.Input(shape=(224,224,3))


- `x`: representa os dados intermediários do processamento
- `training=False`: usdo para impedir instabilidades no treinamento ou destruição do conhecimento prévio do modelo pré-treinado. 

In [44]:
x = base_model(inputs, training=False)

Reduz a dimensionabilidade das características extraídas  pelo ResNet:
- A rede neural desenha vários mapas das características da imagem
- O `GlobalAveragePooling2D` tira a média de cada um destes mapas
- Reduz o número de parâmetros, melhora a generalização e diminui overfitting

In [45]:
x = GlobalAveragePooling2D()(x)

É uma camada de decisão
- Combina as características resumidas pelo `GlobalAveragePooling2D`, aprende as relações
- 128 indica o número de neurônios pelos quais as informações pasarão
  - números menores fazem o modelo decidir muito rápido, sem "raciocinar" direito
  - números maiores podem tornar o modelo pesado demais, além de poderem fazer ele "se confundir"
- O ReLU (Rectified Linear Unit) age como um filtro de relevância
  - valores negativos são transformados em zero - informações
  - valores ppositivos são mantidos

In [46]:
x = Dense(128, activation="relu")(x)

Outra camada de decisão
- O valor 1 indica que deve haver apenas uma saída
- O `sigmoid` faz o modelo decidir entre uma resposta binária
  - 0.34 vira 0
  - 0.73 vira 1

In [47]:
outputs = Dense(1, activation="sigmoid")(x)

Informa ao modelo quais tipos de dados serão sua entrada e o tipo de saída esperada após o processamento, ele automaticamente conecta todas as epatas itermediárias

In [48]:
model = Model(inputs, outputs)

## Compilação do Modelo

- `optimizer` indica ao modelo como ajustar seus erros
  - Adam é um otimizador rápido e estável, bom para a maioria dos problemas
- `loss` é uma forma de medir o quão longe o modelo estava da resosta certa
  - `binary_crossentropy` é usada quando há duas classes e a saída é uma probabilidade entre 0 e 1
  - Pune mais erros confiantes que erros "em dúvida"
- `metrics` permite a acompanhar o desempenho do modelo, se ele está realmente aprendendo
  - `accuracy` é basicamente a precisão do modelo, quantas previsões ele acertou

In [49]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Treinamento do modelo

Indica ao modelo o conjunto de treinamento, de validação e o número de épocas (quantas vezes o modelo vê todo o conjunto de dados)

In [50]:
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 103s 527ms/step - accuracy: 0.7439 - loss: 0.5438 - val_accuracy: 0.5000 - val_loss: 0.9990
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 12s 74ms/step - accuracy: 0.7742 - loss: 0.4582 - val_accuracy: 0.5625 - val_loss: 1.0368
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.7991 - loss: 0.4101 - val_accuracy: 0.5625 - val_loss: 1.0473
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.8135 - loss: 0.3839 - val_accuracy: 0.5625 - val_loss: 1.0653
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 12s 76ms/step - accuracy: 0.8246 - loss: 0.3667 - val_accuracy: 0.6250 - val_loss: 1.0745
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.8313 - loss: 0.3525 - val_accuracy: 0.6250 - val_loss: 1.0427
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.8361 - loss: 0.3423 - val_accuracy: 0.6250 - val_loss: 1.0178
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.8447 - loss: 0.3298 - val

## Salvando o modelo em disco 

Salva o modelo já treinado

In [51]:
model.save(MODELS_PATH)

Salva os pesos atribuídos pelo modelo

In [52]:
model.save_weights(MODEL_WEIGHTS_PATH)

Salva o histórico do treinamento como um arquivo no formato JSON

In [53]:
history_dict = history.history

with open(RESULT_PATH, "w") as f:
    json.dump(history_dict, f)